# 2023_08_17 

### threshold result

threshold 값에 따라서 confusion matrix의 성능이 다르게 나옴  
모델의 예측을 할때 최적의 treshold 값을 찾은 이후에 confusion matrix 계산  
TP값이 더이상 증가하지 않은 threshold를 찾은 이후에 이 값을 기준으로 다시 TP값이 증가하지 않는 threshold를 찾음  
```python 

def plot_metrics_vs_thresholds(decoder_real, pred_raw, idx_df, thresholds, csv_dir, lr):
    data_to_plot = {col: [] for col in ["FP", "TN", "TP", "FN"]}
    previous_tp = -1
    previous_threshold = None
    fine_tuned_thresholds = None

    for threshold in thresholds:
        metric_df = calculate_metrics(decoder_real, pred_raw, idx_df, threshold=threshold)
        cols_to_select = ["FP", "TN", "TP", "FN"]
        col_indices = [metric_df.columns.get_loc(col) for col in cols_to_select]
        result = metric_df.iloc[15, col_indices]
        current_tp = result["TP"]

        if current_tp == previous_tp and previous_threshold is not None:
            # TP가 더 이상 증가하지 않으면 이 threshold와 이전 threshold 사이에서 미세 조정
            fine_tuned_thresholds = np.linspace(previous_threshold, threshold, 20)
            break

        previous_tp = current_tp
        previous_threshold = threshold
        
        for col, value in zip(cols_to_select, result.to_list()):
            data_to_plot[col].append(value)

    # 미세 조정된 threshold 값들에 대해서도 metric 계산
    if fine_tuned_thresholds is not None:
        print(fine_tuned_thresholds)
        previous_tp = -1  # 이 부분을 초기화
        optimal_threshold = None

        for threshold in fine_tuned_thresholds:
            metric_df = calculate_metrics(decoder_real, pred_raw, idx_df, threshold=threshold)
            result = metric_df.iloc[15, col_indices]
            current_tp = result["TP"]

            if current_tp == previous_tp and optimal_threshold is None: 
                # 여기서 첫 번째로 TP 값이 증가하지 않는 threshold를 찾습니다.
                optimal_threshold = threshold

            previous_tp = current_tp

            for col, value in zip(cols_to_select, result.to_list()):
                data_to_plot[col].append(value)

        if optimal_threshold is not None:
            save_dir = f'{csv_dir}/{lr}/'
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            save_path = f'{save_dir}/{lr}/{threshold}.csv'
            metric_df.to_csv(save_path)
            
    fig = go.Figure()
    for metric_name, values in data_to_plot.items():
        fig.add_trace(go.Bar(
            x=fine_tuned_thresholds,
            y=values,
            name=metric_name,
            text=values,
            textposition='auto'
        ))
    fig.update_layout(
        barmode='group', 
        title="Metrics for Different Thresholds",
        xaxis_title='Threshold', 
        yaxis_title='Value'
    )
    fig.update_xaxes(tickvals=fine_tuned_thresholds , tickangle=45)
    fig.write_html(f'{csv_dir}/{lr}/{threshold}.html')
    return fig
```    

> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Long%20Beach_5/output_sacler.ipynb#scaling-%EC%A1%B0%EC%A0%88-%ED%9B%84-plot

위 노트북을 봤을때 threshold를 일괄적용하기에는 무리가 있음  
tft 모델에다가 classifier를 추가해서 treshold값을 따로 계산 할 필요가 있음  
tft 예측이후에 rescaling을 통해서 원래의 값을 복원 후 여기에다가 classifier를 추가함  

> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Long%20Beach_5/result_debug.ipynb#scaling-check